---

# Assignment 1
Welcome to the first assignment! Here you will implement User-User Collaborative Filtering. We will use the MovieLens 20M dataset.

You will write and execute your code in Python using this Jupyter Notebook.

**PREREQUISITE:** Download the MovieLens 20M dataset from <https://grouplens.org/datasets/movielens/20m/>. Extract the contents and look up `ratings.csv`, which is what we'll be working with.

**TASK:** Your job is to *fill in the missing code* only. The place to enter your code is clearly marked with comments.

**SUBMISSION:** You will submit this Notebook via TUWEL (wihtout the movielens folder!).

**GRADING:** We will test whether your code produces the expected output. Therefore hidden tests will compare results of the standard solution with yours (based on the whole dataset - multiple, randomly selected inputs - accuracy of the sulution must be within two decimal places). **Important!** Remove the command "ratings_raw = pd.read_csv(SOME LOCATION OF YOUR CHOICE)" before submitting.

Note that there is a variable `DEBUG` set to `True` for debugging/testing purposes. This creates a small, more manageable subset of the data to work with.

## Preparation

### Import modules
Import python modules we will be using. Generally you should not import additional modules.

In [ ]:
import csv
import pandas as pd
import numpy as np
from scipy import sparse as sp
from scipy.sparse.linalg import norm
import sklearn.preprocessing as pp

Set some formatting options.

In [ ]:
np.set_printoptions(threshold=500, precision=4)
pd.options.display.max_seq_items = 100

### Read the data
Load data from csv file `ratings.csv`. The result is stored in a `pandas.DataFrame` called `ratings_raw`.

In [ ]:
# ratings_raw = pd.read_csv(SOME LOCATION OF YOUR CHOICE) #REMOVE this command before submitting

### Truncate the data

For debugging, let's work with a *truncated* version of the data, containing up to 10000 users and 1000 movies. 

In [ ]:
DEBUG = True

In [ ]:
if DEBUG: 
    ratings_raw = ratings_raw[ (ratings_raw['userId'] < 10000) & (ratings_raw['movieId'] < 1000) ]

Let's see how the data looks like.

In [ ]:
display(ratings_raw.head())

### Preprocess the data
Make sure that movies and users have consecutive indexes starting from 0. Also drop the timestamp column.

The resulting "cleaned" data are stored in `ratings`.

In [ ]:
movieIds = ratings_raw.movieId.unique()
movieIds.sort()
userIds = ratings_raw.userId.unique()
userIds.sort()

m = userIds.size
n = movieIds.size
numRatings = len(ratings_raw)

print ("There are", m, "users,", n, "items and", numRatings, "ratings.")


## movies and users should have consecutive indexes starting from 0
movieId_to_movieIDX = dict(zip(movieIds, range(0, movieIds.size)))
movieIDX_to_movieId = dict(zip(range(0, movieIds.size), movieIds))

userId_to_userIDX = dict(zip(userIds, range(0, userIds.size )))
userIDX_to_userId = dict(zip(range(0, userIds.size), userIds))

## drop timestamps
ratings = pd.concat([ratings_raw['userId'].map(userId_to_userIDX), ratings_raw['movieId'].map(movieId_to_movieIDX), ratings_raw['rating']], axis=1)
ratings.columns = ['user', 'item', 'rating']

display(ratings.head())

### Create the Ratings Matrix

We will convert the `ratings` `DataFrame` into a **Ratings Matrix**. Because it is very sparse, we will use the `scipy.sparse` module to efficiently store and access it.

Specifically, we will create two versions of the same ratings matrix:
- `R` is our basic matrix and is optimized for dot products, which will be useful when computing user-user similarities; `R` is stored in the Compressed Sparse Row format (`csr_matrix`).
- `R_dok` is a different view of the ratings matrix, which allows to quickly test whether a user-item rating exists; `R_dok` is stored in the Dictionary Of Keys format (`dok_matrix`).

In [ ]:
R = sp.csr_matrix((ratings.rating, (ratings.user, ratings.item)))
R_dok = R.todok()

In [ ]:
m = R.shape[0]
n = R.shape[1]
numRatings = R.count_nonzero()

print("There are", m, "users,", n, "items and", numRatings, "ratings.")

## The fun starts here!

### User Average Ratings

The following code computes the average rating of each user. This will be used for mean-centering, i.e., when computing similarities, as well as for making predictions.

In [ ]:
user_sums = R.sum(axis=1).A1 ## matrix converted to 1-D array via .A1
user_cnts = (R != 0).sum(axis=1).A1
user_avgs = user_sums / user_cnts
print("user_avgs", user_avgs)

### User-User Similarity --- TO EDIT

The following function computes the mean-centered cosine similarity between two users.

*Tricks* that might be useful:

To subtract a scalar value `y` from all nonzero entries of a sparse vector `x`, do:
```
x.data = x.data - y
```

The dot product of a sparse vector `x` to sparse vector `y` is:
```
x.dot(y.T)
```

The norm of a sparse vector `x` is:
```
norm(x)
```


If a sparse vector `x` has only a single item, you can access it by:
```
x.A.item()
```

Note that `x.A` returns the dense representation of sparse vector `x`.

In [ ]:
def compute_pairwise_user_similarity(u_id, v_id):
    u = R[u_id,:].copy()
    v = R[v_id,:].copy()
    
    # YOUR CODE HERE
    raise NotImplementedError()
    
    if denominator == 0:
        similarity = 0.;
    else:
        similarity = numerator/denominator
    
    return similarity

In [ ]:
if DEBUG:
    print(compute_pairwise_user_similarity(2, 6))


**DEBUG:** For the truncated dataset, the following should output ~ `0.03585`.

### User to all Users Similarities --- TO EDIT

The following function computes the mean-centered cosine similarities of a given user to all other users.

You should use the `compute_pairwise_user_similarity` function defined above.


You will get a **bonus point**, if you can avoid the for loop and not invoke `compute_pairwise_user_similarity`. The idea is to obtain a copy, say `R_copy`, of matrix `R` that has its rows mean-centered and normalized. This way the given user can be represented by a mean-centered and normalized vector `u`. Then, to obtain the similarity of the user to all others, one needs to take the dot product `R_copy.dot(u.T)`. 

In [ ]:
def compute_user_similarities(u_id):
    uU = np.empty((m,))

    # for v_id in range(m):

    # YOUR CODE HERE
    raise NotImplementedError()
    
    return uU

In [ ]:
if DEBUG:
    uU = compute_user_similarities(2)
    print(uU[6])
    

**DEBUG:** For the truncated dataset, the following should again output ~ `0.03585`.

### Create User Neighborhood --- TO EDIT

The following function creates the user neighborhood of a given user. It takes as input, the target user `u_id` and the target item`i_id`, and uses additional parameters, the size `k` of the neighborhood, and a flag `with_abs_sim`.

If `with_abs_sim` is `True`, the neighborhood should contain up to `k` users with the highest absolute similarity to the target user `u_id`.

If `with_abs_sim` is `False`, the neighborhood should contain up to `k` users with the highest similarity to the target user `u_id`.

The output of the function is `nh`, a `Dictionary` containing key-value entries of the form `v_id : sim(u_id, v_id)`, where `v_id` is another user and `sim(u_id, v_id)` is the similarity between `u_id` and `v_id`.

**Note:** The neighborhood of the target user should not contain itself, i.e., `u_id`, and only include users that have rated the target item `i_id`.


*Tricks* that might be useful:

`np.absolute(x)` returns an array containing the absolute values of each element in array `x`.

`np.argsort(x)` returns an array with the indices that sort array `x` in *increasing* order.

`x[::-1]` returns the reversed array of `x`. So, `np.argsort(x)[::-1]` contains the indices that sort x in *decreasing* order.

To check if user `u_id` has rated item `i_id`, the `R_dok` view of the ratings matrix is helpful:
```
(u_id, i_id) in R_dok
```

In [ ]:
## default values
k = 5
with_abs_sim = False

def create_user_neighborhood(u_id, i_id):
    nh = {} ## the neighborhood dict with (user id: similarity) entries
    ## nh should not contain u_id and only include users that have rated i_id; there should be at most k neighbors
    uU = compute_user_similarities(u_id)
    uU_copy = uU.copy() ## so that we can modify it, but also keep the original
    
    # YOUR CODE HERE
    raise NotImplementedError()
    
    return nh

In [ ]:
if DEBUG:
    k = 5
    with_abs_sim = False
    nh = create_user_neighborhood(0, 8)
    print("with_abs_sim", with_abs_sim, nh)
    with_abs_sim = True
    nh = create_user_neighborhood(0, 8)
    print("with_abs_sim", with_abs_sim, nh)
    

**DEBUG:** For the truncated dataset, the following should output sth like:
```
with_abs_sim False {3439: 0.22132669799727464, 567: 0.21578253384096696, 8084: 0.190930997689841, 7883: 0.18261506018058074, 5352: 0.1798009794567789}
with_abs_sim True {4850: -0.22736427873770565, 3439: 0.22132669799727464, 567: 0.21578253384096696, 8084: 0.190930997689841, 7883: 0.18261506018058074}
```

### Predict a Rating --- TO EDIT

The following function predicts the rating user `u_id` would give to item `i_id`. It uses the flag `with_deviations` to make the prediction.

If `with_deviations` is `True`, the prediction is made over *rating deviations*:
$$ s(u,i) = \overline{r_u} + \frac{\sum_{v \in N(u;i)}w_{uv} (r_{vi}-\overline{r_v})}{\sum_{v \in N(u;i)} |w_{uv}|} .$$

If `with_deviations` is `False`, the prediction is made directly over ratings:
$$ s(u,i) = \frac{\sum_{v \in N(u;i)}w_{uv} r_{vi}}{\sum_{v \in N(u;i)} |w_{uv}|} .$$

The output of the function is the predicted rating `prediction`.

In [ ]:
## a default value
with_deviations = True

def predict_rating(u_id, i_id):
    
    if (u_id, i_id) in R_dok:
        print("user", u_id, "has rated item", i_id, "with", R[u_id, i_id])
    else:
        print("user", u_id, "has not rated item", i_id)
    print("k:", k, "with_deviations:", with_deviations, "with_abs_sim:", with_abs_sim)
    
    
    nh = create_user_neighborhood(u_id, i_id)
    
    neighborhood_weighted_avg = 0.

    # YOUR CODE HERE
    raise NotImplementedError()
    
    if with_deviations:
        prediction = user_avgs[u_id] + neighborhood_weighted_avg
        print("prediction ", prediction, " (user_avg ", user_avgs[u_id], " offset", neighborhood_weighted_avg, ")", sep="")
    else:
        prediction = neighborhood_weighted_avg
        print("prediction ", prediction, " (user_avg ", user_avgs[u_id], ")", sep="")
        
    return prediction

In [ ]:
if DEBUG:
    k = 50
    with_abs_sim = True
    with_deviations = False
    predict_rating(0, 8)
    with_deviations = True
    predict_rating(0, 8)
    

**DEBUG:** For the truncated dataset, the following should output sth like:
```
user 0 has not rated item 8
k: 50 with_deviations: False with_abs_sim: True
prediction 2.3543500356001195 (user_avg 3.6666666666666665)
user 0 has not rated item 8
k: 50 with_deviations: True with_abs_sim: True
prediction 3.306360467515388 (user_avg 3.6666666666666665 offset-0.36030619915127843)
```

## Tests

For this part, **use the full dataset**, and not the truncated one.

In [ ]:
k = 50
with_deviations = True
with_abs_sim = True
predict_rating(0, 8)
%time predict_rating(22, 35)

print("done!")

**Expected output:**
```
user 0 has not rated item 8
k: 50 with_deviations: True with_abs_sim: True
prediction 3.042709922016805 (user_avg 3.742857142857143 offset-0.700147220840338)
user 22 has not rated item 35
k: 50 with_deviations: True with_abs_sim: True
prediction 4.2453310322831594 (user_avg 3.9411764705882355 offset0.30415456169492433)
Wall time: 544 ms
done!

```



In [ ]:
k = 50
with_deviations = False
with_abs_sim = False
predict_rating(0, 12)
%time predict_rating(22, 35)

print("done!")

**Expected output:**
```
user 0 has not rated item 12
k: 50 with_deviations: False with_abs_sim: False
prediction 3.297532757797329 (user_avg 3.742857142857143)
user 22 has not rated item 35
k: 50 with_deviations: False with_abs_sim: False
prediction 3.941867781560111 (user_avg 3.9411764705882355)
Wall time: 547 ms
done!

```

In [ ]:
# feel free to use this field for additional tests

In [ ]:
# feel free to use this field for additional tests

In [ ]:
# feel free to use this field for additional tests

In [ ]:
# feel free to use this field for additional tests

In [ ]:
# feel free to use this field for additional tests